In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import re, sys, random, glob, json, unidecode, unicodedata
import ipdb, time, sys, os, pickle
from IPython.core.display import display, HTML
from IPython.display import clear_output
from urllib.request import urlopen
from collections import Counter
from bs4 import BeautifulSoup  

display(HTML("<style>.container { width:100% !important; }</style>"))

# Matplotlib Configuration
pylab.rcParams.update({'legend.fontsize': 'medium',
                       'font.size'      : 12,
                       'figure.figsize' : (10, 10),
                       'axes.labelsize' : 'medium',
                       'axes.titlesize' : 'medium',
                       'axes.grid'      : 'on',
                       'xtick.labelsize': 'medium',
                       'ytick.labelsize': 'medium'})

def print_runtime(start: float, printer: bool = True) -> None:
    end = time.time()
    if printer:
        print(f'Runtime: {int((end - start) // 60)} min {int((end - start) % 60):2d} sec')
    else:
        return f'({int((end - start) // 60)} min {int((end - start) % 60):2d} sec)'

/var/folders/jf/qykqz6ks0vs56cb_zyf_7x_00000gn/T/ipykernel_20515/597161407.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


---
# Extract names from emails

In [3]:
from openai import OpenAI
client = OpenAI(organization='org-h6gnsCIC5OmHGcNVZm4Z875d')

def compute_sequence_prob(response, alpha=0.7):
    logprob = []
    for item in response.choices[0].logprobs.content:
        logprob.append(item.logprob)

    avg_logprob = sum(logprob) / len(logprob) ** .7
    return np.exp(avg_logprob)

def get_prompt(first_name, email, bio):
    prompt = f"""
    First Name: {first_name}
    Email: {email}
    Bio: {bio}
    """
    return prompt

def get_response(model, sys_message, prompt):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_message},
            {"role": "user", "content": prompt}
        ],
        temperature=0.01,
        logprobs=True,
        top_logprobs=5
    )
    
    prob = compute_sequence_prob(response)
    
    return response, prob


In [4]:
fname = 'First Name Change - Interexo 1_29_24.xlsx - Data - No Change.csv'
df = pd.read_csv(fname)
df.insert(1, 'likelihood', [0]*len(df))
df.rename({'Given Name': 'Preferred Name'}, axis=1, inplace=True)
df = df.astype({'Preferred Name':str, 'likelihood':float})
df.head()

,Preferred Name,likelihood,First Name,Last Name,Email,Company Name,Bio,Address,Country,State,City,Zip Code,Phone #,Phone #.1,Website,Profile Urls
0,nan,0.0,Christopher,Cheung,chris.cheung@fbxbrokers.com,Florida Business Exchange,Chris Cheung is a \t\t\t\t\t\tTampa business b...,550 Reo St.\t\t\t\t\t\t\t\t\t\t\t\t\tSuite 300,United States,FL,Tampa,33602,813-785-9089,NaN,https://fbxbrokers.com,https://brokers.interexo.com/united-states_flo...
1,nan,0.0,William,Gaines,bill@franpro.com,Franprocom,William Gaines is a business broker of the we...,Bowie,United States,MD,20715,NaN,443-583-3070,NaN,https://franpro.com,https://brokers.interexo.com/united-states_mar...
2,nan,0.0,Anthony,Shard,tony@shardmb.com,Shard Marketing,Anthony Shard is a \t\t\t\t\t\tCortez business...,12325 Baypointe Ter,United States,FL,Cortez,34215-2566,810-577-5291,NaN,https://shardmb.com,https://brokers.interexo.com/united-states_flo...
3,nan,0.0,Antonio,Bardy,tonybardy@gmail.com,Florida Connexion Properties,Antonio Bardy is a \t\t\t\t\t\tWindermere busi...,1961 Westhill Run,United States,FL,Windermere,34786-6230,407-535-3639,NaN,NaN,https://brokers.interexo.com/united-states_flo...
4,nan,0.0,Elizabeth,Berger,liz@lizberger.com,Berger Realty Group,Elizabeth Berger is a Coral Springs business b...,1955 University Dr,United States,FL,Coral Springs,33071,954-344-8222,954-650-7848,https://lizberger.com/,https://brokers.interexo.com/united-states_flo...


In [110]:
start = time.time()
model = "gpt-3.5-turbo"
# model = "gpt-4-0125-preview"

sys_message = f"""Given the values for "First Name", "Email" and "Bio" of a company employee, your job is to predict what his/her Preferred Name is. For instance, a person with the first name "Anthony" might prefer being called "Tony", and the evidence of it could be that his email is "tony_pearson@gmail.com" or in his Bio he might be referred to as Tony instead of Anthony. 
However, if you have no evidence for his "Preferred Name" being different than his "First Name" in the email or the "Bio", then you must have no doubts about outputting the original "First Name" and you must output his "First Name" as is. 
If the email features a common preferred version of a First Name, then you can confidently output the preferred version featured in the email. 
Provide your response as a single word and nothing else. Do not include any other text, or any explanation or any special formatting like using backticks (``). Don't make up a full sentence. DO NOT, I repeat, DO NOT start your response with the word Preferred.

I'm providing the "First Name", "Email", and the "Bio" of the business professional below."""

p = 122
# p = 5182
for idx, row in df[p:p+1].iterrows():
    first_name, email, bio = row['First Name'], row['Email'], row['Bio']
    if type(first_name) is not str: 
        continue
    prompt = get_prompt(first_name, email, bio)
    response, prob = get_response(model, sys_message, prompt)
    content = response.choices[0].message.dict()['content']
    
    if prob <= 0.9:
        print(f"{idx:3d}   first_name:{first_name:12s}-> {content:8s}  prob:{round(prob,5)}   {'✔️' if first_name != content else ''}")
        print("-> prob <= 0.80. Re-trying with gpt-4-0125-preview")
        response, prob = get_response("gpt-4-0125-preview", sys_message, prompt)
        
        content = response.choices[0].message.dict()['content']
    df.loc[idx, 'Preferred Name'] = content
    df.loc[idx, 'likelihood'] = round(prob, 2)
    print(f"{idx:3d}   first_name:{first_name:12s}-> {content:8s}  prob:{round(prob,5)}   {'✔️' if first_name != content else ''}")
    
    compute_sequence_prob(response)


122   first_name:Aleksander  -> Aleksander  prob:0.794   
-> prob <= 0.80. Re-trying with gpt-4-0125-preview
122   first_name:Aleksander  -> Aleksander  prob:0.99162   


In [111]:
print(sys_message)

Given the values for "First Name", "Email" and "Bio" of a company employee, your job is to predict what his/her Preferred Name is. For instance, a person with the first name "Anthony" might prefer being called "Tony", and the evidence of it could be that his email is "tony_pearson@gmail.com" or in his Bio he might be referred to as Tony instead of Anthony. 
However, if you have no evidence for his "Preferred Name" being different than his "First Name" in the email or the "Bio", then you must have no doubts about outputting the original "First Name" and you must output his "First Name" as is. 
If the email features a common preferred version of a First Name, then you can confidently output the preferred version featured in the email. 
Provide your response as a single word and nothing else. Do not include any other text, or any explanation or any special formatting like using backticks (``). Don't make up a full sentence. DO NOT, I repeat, DO NOT start your response with the word Prefer

In [112]:
response.choices[0].logprobs.content[0].top_logprobs


[TopLogprob(token='Ale', bytes=[65, 108, 101], logprob=-0.018150644),
 TopLogprob(token='A', bytes=[65], logprob=-4.018151),
 TopLogprob(token='Alex', bytes=[65, 108, 101, 120], logprob=-14.14315),
 TopLogprob(token='\n', bytes=[10], logprob=-16.9244),
 TopLogprob(token='Alexander', bytes=[65, 108, 101, 120, 97, 110, 100, 101, 114], logprob=-19.01815)]

In [94]:
print(sys_message)

Given the values for "First Name", "Email" and "Bio" of a company employee, your job is to predict what his/her Preferred Name is. For instance, a person with the first name "Anthony" might prefer being called "Tony", and the evidence of it could be that his email is "tony_pearson@gmail.com" or in his Bio he might be referred to as Tony instead of Anthony. 
However, if you have no evidence for his "Preferred Name" being different than his "First Name" in the email or the "Bio", then you must have no doubts about outputting the original "First Name" and you must output his "First Name" as is. 
If the email features a common preferred version of a First Name, then you can confidently output the preferred version featured in the email. 
Provide your response as a single word and nothing else. Do not include any other text, or any explanation or any special formatting like using backticks (``). Don't make up a full sentence. DO NOT, I repeat, DO NOT start your response with the word Prefer

In [97]:
response.choices[0].logprobs.content[0].top_logprobs


[TopLogprob(token='Ale', bytes=[65, 108, 101], logprob=-0.64172864),
 TopLogprob(token='Alex', bytes=[65, 108, 101, 120], logprob=-0.7877419),
 TopLogprob(token='Preferred', bytes=[80, 114, 101, 102, 101, 114, 114, 101, 100], logprob=-4.996587),
 TopLogprob(token='The', bytes=[84, 104, 101], logprob=-5.3926516),
 TopLogprob(token='S', bytes=[83], logprob=-6.163844)]

In [92]:
np.exp((response.choices[0].logprobs.content[0].top_logprobs[0]).logprob)

0.7255855173000226

In [18]:
df = pd.read_csv('preferred_names_2.csv', index_col=None)
df.drop(axis=0, inplace=True, columns='Unnamed: 0')
df

,Preferred Name,likelihood,First Name,Last Name,Email,Company Name,Bio,Address,Country,State,City,Zip Code,Phone #,Phone #.1,Website,Profile Urls
0,Chris,1.00,Christopher,Cheung,chris.cheung@fbxbrokers.com,Florida Business Exchange,Chris Cheung is a \t\t\t\t\t\tTampa business b...,550 Reo St.\t\t\t\t\t\t\t\t\t\t\t\t\tSuite 300,United States,FL,Tampa,33602,813-785-9089,NaN,https://fbxbrokers.com,https://brokers.interexo.com/united-states_flo...
1,Bill,1.00,William,Gaines,bill@franpro.com,Franprocom,William Gaines is a business broker of the we...,Bowie,United States,MD,20715,NaN,443-583-3070,NaN,https://franpro.com,https://brokers.interexo.com/united-states_mar...
2,Tony,1.00,Anthony,Shard,tony@shardmb.com,Shard Marketing,Anthony Shard is a \t\t\t\t\t\tCortez business...,12325 Baypointe Ter,United States,FL,Cortez,34215-2566,810-577-5291,NaN,https://shardmb.com,https://brokers.interexo.com/united-states_flo...
3,Tony,1.00,Antonio,Bardy,tonybardy@gmail.com,Florida Connexion Properties,Antonio Bardy is a \t\t\t\t\t\tWindermere busi...,1961 Westhill Run,United States,FL,Windermere,34786-6230,407-535-3639,NaN,NaN,https://brokers.interexo.com/united-states_flo...
4,Liz,0.99,Elizabeth,Berger,liz@lizberger.com,Berger Realty Group,Elizabeth Berger is a Coral Springs business b...,1955 University Dr,United States,FL,Coral Springs,33071,954-344-8222,954-650-7848,https://lizberger.com/,https://brokers.interexo.com/united-states_flo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5319,Yanyu,0.95,Yanyu,Zhang,yzhang@tworld.com,Transworld Business Advisors,"As an agent with Transworld Business Advisors,...",NaN,United States,NaN,NaN,NaN,917-480-7700,800-205-7974,https://www.tworld.com/,https://brokers.interexo.com/united-states/com...
5320,Zabel,0.99,Zabel,Thur,zabel@jonesaa.com,A Jones & Associates,Zabel Thur De Koos is a Gainesville business b...,2815 Nw 13Th Ste 200,United States,FL,Gainesville,32609,352-376-6600,352-215-8713,https://jonesaa.com/,https://brokers.interexo.com/united-states_flo...
5321,Zaheer,0.98,Zaheer,Ahmed,zaheer@saharabusinessbrokers.com,Sahara Business Brokers,ZAHEER AHMED is a Bethesda business broker of ...,10411 Motor City Drive,United States,MD,Bethesda,20817,240-498-0665,NaN,https://saharabusinessbrokers.com,https://brokers.interexo.com/united-states_mar...
5322,Ziggy,0.99,Ziggy,Marszalek,ziggy@bizbrokeragehub.com,Bizbrokerage Hub,Ziggy Marszalek is a Manhasset business broker...,1129 Northern Blvd,United States,NY,Manhasset,11030,516-364-2641,NaN,https://bizbrokeragehub.com,https://brokers.interexo.com/united-states_new...


In [53]:
import math 

for i, (col,row) in enumerate(df[['Preferred Name', 'First Name', 'Bio']].iterrows()):
    if type(row[0]) ==str and row[0] != row[1]:
        print(f'----------------------------------------------\n{i}')
        print(f'{row[0]:10s}  {row[1]:8s}')
        if type(row[2]) != str: continue
        print(row[2].strip('\t'))

----------------------------------------------
0
Chris       Christopher
Chris Cheung is a 						Tampa business broker of the well-known  Florida Business Exchange  business brokerage company who helps people sell their business in 						Tampa, FL and nearest regions.
----------------------------------------------
1
Bill        William 
William Gaines is a  business broker of the well-known  Franprocom business brokerage company who helps people sell their business in MD and nearest regions. Connecting qualified buyers to real business opportunities.
----------------------------------------------
2
Tony        Anthony 
Anthony Shard is a 						Cortez business broker of the well-known  Shard Marketing business brokerage company who helps people sell their business in 						Cortez, FL and nearest regions.
----------------------------------------------
3
Tony        Antonio 
Antonio Bardy is a 						Windermere business broker of the well-known  Florida Connexion  Properties business broke

In [58]:
df.iloc[122]

Preferred Name                                                 Akey
likelihood                                                      1.0
First Name                                               Aleksander
Last Name                                                 Keylikhes
Email                                         akeylikhes@tworld.com
Company Name                      Transworld Business Brokers Miami
Bio               Aleksander Keylikhes is a Miami business broke...
Address                                8323 Nw 12 Street, Suite 104
Country                                               United States
State                                                            FL
City                                                          Miami
Zip Code                                                      33126
Phone #                                                305-949-1199
Phone #.1                                              305-799-0729
Website                                         